In [29]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

os.chdir(r"C:\Users\cgree\Documents\Astra\Space_weather5_22\weakley_all")

In [30]:
df = pd.read_csv('combined_csv.csv')
df.index = pd.to_datetime(df[['year','month','day' ,'hour','minute','second']])
df = df.drop(columns=["year", "month", "day", "hour", "minute", "second"])

In [31]:
df3 = pd.read_csv(r'C:\Users\cgree\Documents\Astra\Space_weather5_22\2018_sm_reg.csv')
df3 = df3.set_index('Date_UTC')

In [32]:
dfx = df.join(df3, how='inner')#  how='left')   #'inner')

In [33]:
dfx= dfx.replace([99999.9, 9999.99, 999.99, 999999.0 ,999999.00,-99999.990000, '#REF!'], np.nan)

In [34]:
del dfx['AE']
def calculate_derivative(df, new_column_name, column_to_derive, shift_in_date = 1):
    df[new_column_name] = (df[column_to_derive])-(df[column_to_derive].shift(1))/(shift_in_date)
    return df[new_column_name]

new_column_name = 'FOD' #first order derivative
column_to_derive = 'gic'
calculate_derivative(dfx, new_column_name, column_to_derive, shift_in_date = 1)

#second order derivative
new_column_name = 'SOD' #second order derivative
column_to_derive = 'FOD'
calculate_derivative(dfx, new_column_name, column_to_derive, shift_in_date = 1)

2018-03-01 00:00:00     NaN
2018-03-01 00:01:00     NaN
2018-03-01 00:02:00   -0.07
2018-03-01 00:03:00    0.03
2018-03-01 00:04:00    0.04
                       ... 
2018-03-31 23:55:00    0.12
2018-03-31 23:56:00   -0.04
2018-03-31 23:57:00   -0.10
2018-03-31 23:58:00    0.21
2018-03-31 23:59:00   -0.26
Name: SOD, Length: 44579, dtype: float64

In [36]:
data = dfx[['gic', 'FOD', 'SOD']]
data = data[2:] #dropping first two rows with Nan

In [38]:
date_split = '2018-03-15 00:00:00'
# train = data[:date_split]
# test = data[date_split:]
train = data['2018-03-01 00:00:00':'2018-03-15 00:00:00']
test = data['2018-03-15 01:00:00':'2018-03-31 23:59:00']
len(train), len(test)

(20098, 24420)

In [46]:
class Environment1:
    
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
#         self.reset()
        
        self.range = 10  # +/- value the randomly select number can be between
        self.bounds = 20  # Action space bounds

        self.action_space = spaces.Discrete(3) #the number of options we have to pick from up/down/stay the same- 0 or 1
        self.observation_space = spaces.Box(np.array([0, 0, 0, 0, 0]), np.array([10, 10, 10, 10, 10]))
        
#         self.action_space = spaces.Box(low=np.array([-self.bounds]), high=np.array([self.bounds]),
#                                        dtype=np.float32)
#         self.observation_space = spaces.Discrete(3) #the number of options we have to pick from up/down/stay the same

        self.number = 0
        self.guess_count = 0
        self.guess_max = 200
        self.observation = 0

        self.seed()
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    
    def step(self, action):
        #assert self.action_space.contains(action)

        if action < self.number:
            self.observation = 1

        elif action == self.number:
            self.observation = 2

        elif action > self.number:
            self.observation = 3

        reward = ((min(action, self.number) + self.bounds) / (max(action, self.number) + self.bounds)) ** 2

        self.guess_count += 1
        done = self.guess_count >= self.guess_max

        return self.observation, reward[0], done, {"number": self.number, "guesses": self.guess_count}


    
#     def step(self, act):
#         reward = 0
        
#         # act = 0: stay, 1: up(left), 2: down(right)
#         if act == 1: #up
#             self.positions.append(self.data.iloc[self.t, :]['gic'])
#         elif act == 2: # down
#             if len(self.positions) == 0:
#                 reward = - 1
#             else:
#                 profits = 0
#                 for p in self.positions:
#                     profits += 1 #(self.data.iloc[self.t, :]['gic'] - p)
#                 reward += profits
#                 self.profits += profits
#                 self.positions = []
        
#         # set next time
#         self.t += 1
#         self.position_value = 0
#         for p in self.positions:
#             self.position_value += (self.data.iloc[self.t, :]['gic'] - p)
#         self.history.pop(0)
#         self.history.append(self.data.iloc[self.t, :]['FOD'])
        
#         # clipping reward
#         if reward > 0:
#             reward = 1
#         elif reward < 0:
#             reward = -1
        
#         return [self.position_value] + self.history, reward, self.done # obs, reward, done

In [47]:
env = Environment1(train)
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print(env.step(pact))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


TypeError: 'float' object is not subscriptable

In [23]:
import numpy as np

import gym
from gym import spaces
from gym.utils import seeding


class HotterColder(gym.Env):
    """Hotter Colder
    The goal of hotter colder is to guess closer to a randomly selected number

    After each step the agent receives an observation of:
    0 - No guess yet submitted (only after reset)
    1 - Guess is lower than the target
    2 - Guess is equal to the target
    3 - Guess is higher than the target

    The rewards is calculated as:
    (min(action, self.number) + self.range) / (max(action, self.number) + self.range)

    Ideally an agent will be able to recognise the 'scent' of a higher reward and
    increase the rate in which is guesses in that direction until the reward reaches
    its maximum
    """
    def __init__(self):
        self.range = 10  # +/- value the randomly select number can be between
        self.bounds = 20  # Action space bounds

        self.action_space = spaces.Discrete(3) #the number of options we have to pick from up/down/stay the same- 0 or 1
        self.observation_space = spaces.Box(np.arrary([0, 0, 0, 0, 0]), np.array([10, 10, 10, 10, 10]))
        
#         self.action_space = spaces.Box(low=np.array([-self.bounds]), high=np.array([self.bounds]),
#                                        dtype=np.float32)
#         self.observation_space = spaces.Discrete(3) #the number of options we have to pick from up/down/stay the same

        self.number = 0
        self.guess_count = 0
        self.guess_max = 200
        self.observation = 0

        self.seed()
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        #assert self.action_space.contains(action)

        if action < self.number:
            self.observation = 1

        elif action == self.number:
            self.observation = 2

        elif action > self.number:
            self.observation = 3

        reward = ((min(action, self.number) + self.bounds) / (max(action, self.number) + self.bounds)) ** 2

        self.guess_count += 1
        done = self.guess_count >= self.guess_max

        return self.observation, reward[0], done, {"number": self.number, "guesses": self.guess_count}

    def reset(self):
        self.number = self.np_random.uniform(-self.range, self.range)
        self.guess_count = 0
        self.observation = 0
        return self.observation

In [24]:
import numpy as np
import gym
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from collections import deque
from keras.layers import Input, Activation, Dense, Flatten, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D
from keras.models import Model
from keras import backend as K
from keras.models import Model,Sequential

In [25]:
class Agent:
    def __init__(self, env):
        self.env = env
        self.input_dim = env.observation_space.shape[0]
        self.output_dim = env.action_space.n
        self.create_model()

    def create_model(self):
        self.model = keras.Sequential()
        self.model.add(Dense(3,input_shape=(1,)))
        self.model.add(Dense(8))
        self.model.add(Activation('relu'))
        self.model.add(Dense(32))
        self.model.add(Activation('relu'))
        self.model.add(Dense(16))
        self.model.add(Activation('relu'))
        self.model.add(Dense(self.output_dim))
        #model.add(Activation('softmax'))
        #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        #self.model = Model(inputs=X, outputs=net)
        self.model.compile( loss='mean_squared_error',optimizer='rmsprop',metrics=['accuracy'])

    def act(self, X, eps=1.0):
        if np.random.rand() < eps:
            return self.env.action_space.sample()

        X = X.reshape(-1, self.input_dim)
        Q = self.model.predict_on_batch(X)
        return np.argmax(Q, 1)[0]

    def train(self, X_batch, y_batch):
        return self.model.train_on_batch(X_batch, y_batch)

    def predict(self, X_batch):
        return self.model.predict_on_batch(X_batch)


def create_batch(agent, memory, batch_size, discount_rate):
    sample = random.sample(memory, batch_size)
    sample = np.asarray(sample)

    s = sample[:, 0]
    a = sample[:, 1].astype(np.int8)
    r = sample[:, 2]
    s2 = sample[:, 3]
    d = sample[:, 4] * 1.

    X_batch = np.vstack(s)
    y_batch = agent.predict(X_batch)

    y_batch[np.arange(batch_size), a] = r + discount_rate * np.max(agent.predict(np.vstack(s2)), 1) * (1 - d)

    return X_batch, y_batch


def print_info(episode, reward, eps):
    msg = f"[Episode {episode:>5}] Reward: {reward:>5} EPS: {eps:>3.2f}"
    print(msg)


def main():
    n_episode = 1000
    discount_rate = 0.99
    n_memory = 5000
    batch_size = 32
    eps = 1.0
    min_eps = 0.1
    #env_name = 'CartPole-v0'
    #env = gym.make(env_name)
    env = HotterColder()
    agent = Agent(env)
    memory = deque()

    # HotterColder Clear Condition
    # Average reward per episode > 195.0 over 100 episodes
    LAST_100_GAME_EPISODE_REWARDS = deque()

    for episode in range(n_episode):
        done = False
        s = env.reset()
        eps = max(min_eps, eps - 1/(n_episode/2))
        episode_reward = 0
        while not done:
            a = agent.act(s, eps)
            s2, r, done, info = env.step(a)
            episode_reward += r

            if done and episode_reward < 200:
                r = -100

            memory.append([s, a, r, s2, done])

            if len(memory) > n_memory:
                memory.popleft()

            if len(memory) > batch_size:
                X_batch, y_batch = create_batch(agent, memory, batch_size, discount_rate)
                agent.train(X_batch, y_batch)

            s = s2

        print_info(episode, episode_reward, eps)
        LAST_100_GAME_EPISODE_REWARDS.append(episode_reward)
        if len(LAST_100_GAME_EPISODE_REWARDS) > 100:
            LAST_100_GAME_EPISODE_REWARDS.popleft()

        if np.mean(LAST_100_GAME_EPISODE_REWARDS) >= 195.0:
            print(f"Game solved in {episode + 1} with average reward {np.mean(LAST_100_GAME_EPISODE_REWARDS)}")

    env.close()


if __name__ == '__main__':
    main()

IndexError: tuple index out of range

In [26]:
import numpy as np
import gym
import random
from gym import spaces
from gym.utils import seeding
from collections import deque
from keras.layers import Input, Activation, Dense, Flatten, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D
from keras.models import Model,Sequential
from keras import backend as K
from keras import optimizers

In [28]:
class Agent:
    def __init__(self, env):
        self.env = env
        self.input_dim = 1
        self.output_dim = env.action_space.shape[0]
        print(self.output_dim)
        self.create_model()

    def create_model(self):
        self.model = Sequential()
        self.model.add(Dense(3,input_shape=(1,)))
        self.model.add(Dense(8))
        self.model.add(Activation('relu'))
        self.model.add(Dense(32))
        self.model.add(Activation('relu'))
        self.model.add(Dense(16))
        self.model.add(Activation('relu'))
        self.model.add(Dense(self.output_dim))
        #model.add(Activation('softmax'))
        #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        #self.model = Model(inputs=X, outputs=net)
        self.model.compile( loss='mean_squared_error',optimizer='rmsprop',metrics=['accuracy'])
        #print(self.model.summary())
        #self.model = model
        #return model

    def act(self, X, eps=1.0):
        if np.random.rand() < eps:
            return self.env.action_space.sample()
        print(self.env.action_space.sample())
        #X = float(X)
        print(X)
        X = np.asarray(X,)
        #print(X)
        X = X.reshape(-1, )
        print(X)
        #print(X.shape)
        #print(self.env.action_space.sample())
        #print(X)
        #X = np.array(X)
        Q = self.model.predict(X)
        print(Q)
        print(np.argmax(Q, 1)[0])
        #return np.argmax(Q, 1)[0]
        return(Q)

    def train(self, X_batch, y_batch):
        return self.model.train_on_batch(X_batch, y_batch)

    def predict(self, X_batch):
        #print(self.model.summary())
        #print(X_batch)
        #time()
        return self.model.predict_on_batch(X_batch)


def create_batch(agent, memory, batch_size, discount_rate):
    sample = random.sample(memory, batch_size)
    sample = np.asarray(sample)

    s = sample[:, 0]
    a = sample[:, 1].astype(np.int8)
    r = sample[:, 2]
    s2 = sample[:, 3]
    d = sample[:, 4] * 1.
    #print(a)
    #time.sleep()
    #print(s)
    X_batch = np.vstack(s)
    print(X_batch)
    y_batch = agent.predict(X_batch)
    #print(np.max(agent.predict(np.vstack(s2))))
    #print(np.arange(batch_size))
    #time.sleep(5)
    #y_batch[np.arange(batch_size), a] = r + discount_rate * np.max(agent.predict(np.vstack(s2)), 1) * (1 - d)
    #print(y_batch)
    #print(r + discount_rate * np.max(agent.predict(np.vstack(s2)), 1) * (1 - d))
    #y_batch[np.arange(batch_size)] = r + discount_rate * np.max(agent.predict(np.vstack(s2)), 1) * (1 - d)

    return X_batch, y_batch


def print_info(episode, reward, eps):
    #print("[Episode {episode:>5}] Reward: {reward:>5} EPS: {eps:>3.2f}")
    print("Episode " + str(episode) + " Reward " + str(reward) + " EPS " + str(eps))


def main():
    n_episode = 1000
    discount_rate = 0.99
    n_memory = 5000
    batch_size = 32
    eps = 1.0
    min_eps = 0.1
    #env_name = 'CartPole-v0'
    #env = gym.make(env_name)
    env = HotterColder()
    agent = Agent(env)
    memory = deque()

    # CartPole-v0 Clear Condition
    # Average reward per episode > 195.0 over 100 episodes
    LAST_100_GAME_EPISODE_REWARDS = deque()

    for episode in range(n_episode):
        done = False
        s = env.reset()
        print("s: " + str(s))
        eps = max(min_eps, eps - 1/(n_episode/2))
        episode_reward = 0
        while not done:
            a = agent.act(s, eps)
            s2, r, done, info = env.step(a)
            episode_reward += r

            if done and episode_reward < 200:
                r = -100

            memory.append([s, a, r, s2, done])

            if len(memory) > n_memory:
                memory.popleft()

            if len(memory) > batch_size:
                X_batch, y_batch = create_batch(agent, memory, batch_size, discount_rate)
                agent.train(X_batch, y_batch)

            s = s2

        print_info(episode, episode_reward, eps)
        LAST_100_GAME_EPISODE_REWARDS.append(episode_reward)
        if len(LAST_100_GAME_EPISODE_REWARDS) > 100:
            LAST_100_GAME_EPISODE_REWARDS.popleft()

        if np.mean(LAST_100_GAME_EPISODE_REWARDS) >= 195.0:
            #print(f"Game solved in {episode + 1} with average reward {np.mean(LAST_100_GAME_EPISODE_REWARDS)}")
            print("Game solved in " + str(episode + 1) + " with average reward  " + str(np.mean(LAST_100_GAME_EPISODE_REWARDS)) )

    env.close()


if __name__ == '__main__':
    main()

1
s: 0
[[1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]]
[[1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [0]
 [1]
 [3]
 [1]
 [1]
 [1]]
[[1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [0]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]]
[[3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]]
[[1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [0]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]]
[[1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [1]
 [0]
 [1]
 [1]
 [1]
 [3]
 [1]]
[[3]
 [1]
 [3]
 [3]
 [1]
 [

[[3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]]
[[3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [0]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]]
[[3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [0]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]]
[[3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]]
[[1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]

 [3]]
[[1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]]
[[1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]]
[[3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [3]
 [3]
 [3]
 [0]
 [3]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]]
[-18.245216]
3
[3]
[[-0.1106

[[3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]]
[[1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]]
[[3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]]
[[1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [0]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]]
[[1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]]
[[1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]]
[[1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]

 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [0]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]]
[[3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]]
[[3]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]]
[[3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]]
[[3]
 [1]
 [3]
 [1]
 [1]
 [3

...

[[-0.11069345]]
0
[[1]
 [3]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]]
[[1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]]
[[1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]]
[[3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]]
[[3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]]
[[1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]]
[[3]
 [3]
 [3]
 

 [1]]
[[3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]]
[[3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]]
[[3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]]
[[3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]]
[[3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [3]
 [1]
 [3

 [1]]
[[1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]]
[[1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]]
[[1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]]
[[1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]]
[[3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]]
[[3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]]
[[1]
 [3]
 [1]
 [1]
 [1]
 [1

[[1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]]
[[1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]]
[[3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]]
[[1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [3]]
[[1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [3]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]]
[-16.922369]
3
[3]
[[-0.11069345]]
0
[[1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [3]
 [1]
 [3

KeyboardInterrupt: 